Short Text about my project 

##### First, we need to install all of our packages, the file 'requirements.txt' holds all of the packages we need here, so we just need to run a simple pip install

In [1]:
!pip install -r requirements.txt

##### Next, we will import the Modules that will be used in this project flow, this is being done in the beginning to avoid any unexpected errors throughout this walkthrough.

In [ ]:
#import matplotlib
#import pandas
#import geopandas
#import folium
#from folium.plugins import HeatMap
#import json
#from glob import glob
#import matplotlib.pyplot as plt
#import folium
#import mapclassify
#import contextily
#from scipy.stats import linregress

# Importing the project files:

##### The first file we will take a look at is the directory of all the public trees in the Agglomeration of Montreal

In [2]:
import pandas as pd 

public_trees = pd.read_csv('arbres-publics.csv', low_memory = False)
 
public_trees.fillna(0, inplace=True)

public_trees

,ARROND_NOM,Emplacement,Coord_X,Coord_Y,SIGLE,DHP,NOM_PARC,Longitude,Latitude
0,Ahuntsic - Cartierville,Parterre Gazonné,287967.933,5043937.611,GLTRSK,25.0,RAIMBAULT,-73.715515,45.535151
1,Ahuntsic - Cartierville,Parterre Gazonné,287961.715,5043945.204,GLTRSK,32.0,RAIMBAULT,-73.715595,45.535219
2,Ahuntsic - Cartierville,Parterre Gazonné,287956.685,5043932.522,GYDI,21.0,RAIMBAULT,-73.715659,45.535105
3,Ahuntsic - Cartierville,Parterre Gazonné,287985.011,5043989.384,FRNIFG,5.0,RAIMBAULT,-73.715298,45.535617
4,Ahuntsic - Cartierville,Parterre Gazonné,287961.737,5043987.310,QURU,34.0,RAIMBAULT,-73.715596,45.535598
...,...,...,...,...,...,...,...,...,...
336368,Saint-Laurent,Îlot de Verdure,291677.894,5043459.985,GYDI,7.0,0,-73.668001,45.530933
336369,Saint-Laurent,Îlot de Verdure,291675.641,5043470.398,GYDI,5.0,0,-73.668030,45.531027
336370,Saint-Laurent,Îlot de Verdure,291667.103,5043492.484,GYDI,5.0,0,-73.668140,45.531225
336371,Saint-Laurent,Îlot de Verdure,291662.713,5043501.640,GYDI,5.0,0,-73.668196,45.531307


##### There is a lot going on in this file, so the *csv* is also available in the notebook which can help you follow along with what exactly is going on.  

First of all, we have the **ARROND_NOM** which is the name of the burough that the tree is situated;

Second, we have **Emplacement,** which is where the tree is placed (Sidewalk, park, etc.)

Next two are the **X and Y coordinates** in the *Projection: Québec Modified Transverse Mercator(NAD83), Québec MTM Zone 8.*

After that, we have the **SIGLE**, this is an Acronym composed of the first two letters of the *genus, species* and *cultivar.*

The **DHP** is the is the *diameter at breast height* , this is the diameter measure of the tree, generally, the trees in my experience will range from 15cm to over 3m (usually really old maple trees), but the trees are typically planted when the diameter reaches around 10cm. 

The **NOM_PARC** is the name of the park if the tree is planted in a park, we will see whether or not we keep this file when we start looking more into our file. 

The last two columns are the **Latitude and Longitude**, both in *WGS 84*. 

#### Lets see how many types of trees exist on the public tree registry:

In [26]:
tree_types = public_trees['SIGLE'].value_counts()

tree_types

ACSA      33217
ACPL      30131
FRPE      19374
TICO      15711
CEOC      10176
          ...  
PRSLEG        1
PRAR          1
ACGA          1
PS            1
MAXXSR        1
Name: SIGLE, Length: 715, dtype: int64

It looks like there are 715 types of trees on the public registry! That is a lot of trees! 
Lets see when this abundance of the same tree ends by using the .head() on this variable

We will look at the top 10 species here:

In [35]:
tree_types.head(10)

ACSA      33217
ACPL      30131
FRPE      19374
TICO      15711
CEOC      10176
GLTRSK     9940
GYDI       8442
GLTRIN     7942
PIPU       6972
ULPU       5934
Name: SIGLE, dtype: int64

Lets see how many types of Maple trees there are! The maple trees, or *Acer,* which represents all maple tree and shrubs that exist. So this means we will be searcging for all of the variables in the SIGLE column that begin with **AC** for *Acer*  

In [47]:
#extract each variable in the SIGLE column that begins with AC for ACER
maple = public_trees[public_trees['SIGLE'].str[:2] == 'AC']

maple = maple['SIGLE'].value_counts()

maple

ACSA      33217
ACPL      30131
ACPLCO     5379
ACSC       4202
ACPLDE     3875
          ...  
ACSCBO        2
ACTRCS        2
ACRUKA        1
ACNEKG        1
ACGA          1
Name: SIGLE, Length: 101, dtype: int64

There are 101 different types and cultivars of Maple trees! Lets see if we can narrow this down to some we know:

In [49]:
maple.head(10)

ACSA      33217
ACPL      30131
ACPLCO     5379
ACSC       4202
ACPLDE     3875
ACFRAB     3435
ACPLEQ     3148
ACSAPY     2369
ACNE       2255
ACRU       2015
Name: SIGLE, dtype: int64

##### So looking at the first 10 we have:

ACSA - Acer Saccharrinium (silver maple)

ACPL - Acer Platanoides (norway maple)

ACSC - Acer Saccharum (sugar maple)

ACFR - Acer x Freemanii (freemanii maple)
*which is the cross of a red maple and sugar maple, very very popular here*

ACNE - Acer Negundo (box-elder or ash-leaved maple) 
*Fun fact about this tree: there is a very common confusion between the ash tree and this tree, and a lot of citizens will call in to cut down what they think is their ash tree, but find out that it is a maple tree when an inspector comes to check it out. I was that inspector for part of a summer stage**

ACRU - Acer Rubrum (red maple)

Some of them have more letters in their acronyms, this isnt super important, it is just the cultivar, so sometimes it is named after the pepiniere it was grown in, or simply the color the leaves may turn at certain times of the year. 

Lets look at all the types of ash trees now, these are the *FR* for **Fraxinus** 

In [54]:
#extract each variable in the SIGLE column that begins with FR for 
ash = public_trees[public_trees['SIGLE'].str[:2] == 'FR']

ash = ash['SIGLE'].value_counts()

ash.count()

43

So it looks like we have 43 types of ash trees, lets look at them all since they will all actually appear when i call this variable:

In [55]:
ash

FRPE      19374
FRPEMS     3264
FRPESU     3036
FRAM       2210
FRPEPA     1380
FR          984
FRAMAP      353
FRAMMA      176
FRNI        144
FRAMKL      121
FRAMNB       65
FRAMSK       35
FRPEPS       35
FRNIFG       21
FRPEHA       21
FRPECI        9
FREX          9
FRNINT        8
FRQU          8
FRMN          6
FRPR          6
FREXAU        5
FRANOX        4
FRCHRH        4
FRPEAU        4
FREXDI        3
FRPA          3
FRPESP        2
FRPESB        2
FROR          2
FRSI          2
FRPEWA        2
FREXEU        1
FREXKI        1
FRBU          1
FRANRA        1
FREXDO        1
FRSO          1
FRANSY        1
FRVE          1
FRAMJU        1
FRLO          1
FREXHE        1
Name: SIGLE, dtype: int64

So for a quick rundown on the important types of Ash Trees; 

FRAM is an american ash tree, or a white ash

FRPE is a pennsylvanian ash tree, or a green ash

FRNI is the black ash tree


Lets look at the next prominent tree on our list from eariler, the *TI* or **Tilia**, which is the basswood/linden tree

In [58]:
linden = public_trees[public_trees['SIGLE'].str[:2] == 'TI']

linden = linden['SIGLE'].value_counts()

linden

TICO      15711
TIAM       3534
TICOGR      782
TIMOHG      545
TICOSH      496
TIAMRE      489
TITO        464
TICOGL      462
TITOST      229
TITOSS      212
TI          156
TIERWR      146
TICOCO      107
TIAMBO       94
TITOBR       56
TIAMLI       47
TICORO       36
TIEREC       25
TIMO         15
TIFLGL       11
TICOMO        9
TIAMDU        8
TIAMFA        7
TIFLDR        5
TICOSS        5
TICOBA        3
TIER          1
TICOGG        1
Name: SIGLE, dtype: int64

So here we have:

TICO which is tilia cordata (little leaf linden, native to europe)

TIAM which is tilia americana (american basswood)

TIMO which is tilia mongolica (mongolian lime, which is actually a smaller tree compared to the other two)

In [3]:
import geopandas as gpd

geo_trees = gpd.GeoDataFrame(public_trees, geometry=gpd.points_from_xy(public_trees.Longitude, public_trees.Latitude))

geo_trees

,ARROND_NOM,Emplacement,Coord_X,Coord_Y,SIGLE,DHP,NOM_PARC,Longitude,Latitude,geometry
0,Ahuntsic - Cartierville,Parterre Gazonné,287967.933,5043937.611,GLTRSK,25.0,RAIMBAULT,-73.715515,45.535151,POINT (-73.71551 45.53515)
1,Ahuntsic - Cartierville,Parterre Gazonné,287961.715,5043945.204,GLTRSK,32.0,RAIMBAULT,-73.715595,45.535219,POINT (-73.71559 45.53522)
2,Ahuntsic - Cartierville,Parterre Gazonné,287956.685,5043932.522,GYDI,21.0,RAIMBAULT,-73.715659,45.535105,POINT (-73.71566 45.53511)
3,Ahuntsic - Cartierville,Parterre Gazonné,287985.011,5043989.384,FRNIFG,5.0,RAIMBAULT,-73.715298,45.535617,POINT (-73.71530 45.53562)
4,Ahuntsic - Cartierville,Parterre Gazonné,287961.737,5043987.310,QURU,34.0,RAIMBAULT,-73.715596,45.535598,POINT (-73.71560 45.53560)
...,...,...,...,...,...,...,...,...,...,...
336368,Saint-Laurent,Îlot de Verdure,291677.894,5043459.985,GYDI,7.0,0,-73.668001,45.530933,POINT (-73.66800 45.53093)
336369,Saint-Laurent,Îlot de Verdure,291675.641,5043470.398,GYDI,5.0,0,-73.668030,45.531027,POINT (-73.66803 45.53103)
336370,Saint-Laurent,Îlot de Verdure,291667.103,5043492.484,GYDI,5.0,0,-73.668140,45.531225,POINT (-73.66814 45.53122)
336371,Saint-Laurent,Îlot de Verdure,291662.713,5043501.640,GYDI,5.0,0,-73.668196,45.531307,POINT (-73.66820 45.53131)


Lets upload the census data.

In [4]:
pop_data = pd.read_csv('MTL_DATA.csv')
 
pop_data.fillna(0, inplace=True)

pop_data

,GEOUID,CDcode,POP2021,POPDENS,LOWINC,LOWINC0-17,LOWINC18-64,LOWINC65+
0,2466,66,2004265,4022.3,213720.0,29585.0,153385.0,30745.0
1,2466007,66,4394,361.6,380.0,80.0,255.0,45.0
2,2466023,66,1762949,4833.5,199665.0,26605.0,144765.0,28295.0
3,2466032,66,19658,4860.9,1860.0,435.0,1130.0,295.0
4,2466047,66,5115,3737.9,245.0,40.0,165.0,40.0
5,2466058,66,34504,4903.7,2740.0,550.0,1620.0,565.0
6,2466062,66,7037,3922.1,380.0,75.0,245.0,55.0
7,2466072,66,20953,2776.7,1380.0,395.0,820.0,165.0
8,2466087,66,19302,923.0,1140.0,175.0,765.0,195.0
9,2466092,66,30,154.7,0.0,0.0,0.0,0.0
